In [3]:
import random
import tensorflow as tf
import numpy as np

from tensorflow.models.rnn import rnn_cell
from tensorflow.models.rnn import seq2seq

In [8]:
MIN_NUMBER = 1
MAX_NUMBER = 99
def number():
    return random.randrange(MIN_NUMBER, MAX_NUMBER + 1)

In [16]:
SOURCE_VOCAB, TARGET_VOCAB = '10*\n', '10\n'
MAX_SOURCE_LEN = 2 + 2 * len(str(MAX_NUMBER))
MAX_TARGET_LEN = 5

# Generates one example (source, target) pair
def generate():
    a, b = number(), number()
    c = a + b
    source, target = '{0:b}+{0:b}\n'.format(a, b), '{0:b}\n'.format(c)
    assert all(ch in SOURCE_VOCAB for ch in source)
    assert all(ch in TARGET_VOCAB for ch in target)
    assert len(source) <= MAX_SOURCE_LEN
    assert len(target) <= MAX_TARGET_LEN
    return source, target

In [12]:
# Set up some hyperparameters & other initialization
learning_rate = tf.Variable(0.5, trainable=False)
learning_rate_decay_op = learning_rate.assign(learning_rate * 0.99)
global_step = tf.Variable(0, trainable=False)

NUM_LAYERS = 2
LAYER_SIZE = 128

In [13]:
# Set up the core RNN cells of the tensor network
single_cell = rnn_cell.BasicLSTMCell(LAYER_SIZE)
multi_cell = rnn_cell.MultiRNNCell([single_cell] * NUM_LAYERS)

In [14]:
# This function handles embedding for the input and attention.
def seq2seq_f(encoder_inputs, decoder_inputs, do_decode):
    return seq2seq.embedding_attention_seq2seq(
        encoder_inputs, decoder_inputs, multi_cell,
        len(SOURCE_VOCAB), len(TARGET_VOCAB),
        feed_previous=do_decode)

In [17]:
# Set up input and weight placeholders, for the source and target embeddings
encoder_inputs = [tf.placeholder(tf.int32,
                                 shape=[None],
                                 name='encoder{0}'.format(i))
                  for i in range(len(SOURCE_VOCAB))]
decoder_inputs = [tf.placeholder(tf.int32,
                                 shape=[None],
                                 name='decoder{0}'.format(i))
                  for i in range(len(TARGET_VOCAB))]
target_weights = [tf.placeholder(tf.float32,
                                 shape=[None],
                                 name='weight{0}'.format(i))
                  for i in range(len(TARGET_VOCAB))]    

In [19]:
# Our targets are decoder inputs shifted by one.
targets = [decoder_inputs[i + 1]
           for i in range(len(decoder_inputs) - 1)]